In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import pygame, time

path_alarm = r"C:\Users\osama\OneDrive\Desktop\Moahmmed Upwork\mixkit-facility-alarm-sound-999.wav"
pygame.init()
pygame.mixer.music.load(path_alarm)

video = cv2.VideoCapture(0)
# video = cv2.VideoCapture(r'')
model = YOLO(r'C:\Users\osama\Downloads\Drowsiness Detection\Drowsiness Detection Using YOLOv8.pt')
class_list = model.names
print(class_list)

awake_time = 0
drowsy_time = 0

start_time_awake = None
start_time_drowsy = None

current_state = None
cls = None

while True:
    _, image = video.read()
    if image is None:
        break
    image = cv2.resize(image, (1280, 640))
    h, w, c = image.shape
    image = cv2.flip(image, 1)

    cv2.putText(image, f'awake(sec): {awake_time}', (10, 50),
                cv2.FONT_ITALIC, 0.5, (0, 165, 255), 1)
    cv2.putText(image, f'drowsy(sec): {drowsy_time}', (10, 70),
                cv2.FONT_ITALIC, 0.5, (0, 165, 255), 1)



    result = model.predict(image)

    a = result[0].boxes.data
    px = pd.DataFrame(a).astype(float)

    for index, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        cls = class_list[int(row[5])]
        confidence = row[4]

        if cls == 'drowsy':
            if current_state != 'drowsy':
                start_time_drowsy = time.time()
                start_time_awake = None
                current_state = 'drowsy'
            drowsy_time = round(time.time() - start_time_drowsy, 3)
            awake_time = 0
        elif cls == 'awake':
            if current_state != 'awake':
                start_time_awake = time.time()
                start_time_drowsy = None
                current_state = 'awake'
            awake_time = round(time.time() - start_time_awake, 3)
            drowsy_time = 0


        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 165, 255), 2)
        cv2.rectangle(image, (x1, y1 - 25), (x1 + 200, y1), (0, 165, 255), -1)

        cv2.putText(image, f'id:{index}', (x1, y1 - 10), cv2.FONT_ITALIC, 0.7, (0, 0, 0), 1)
        cv2.putText(image, f'{cls}', (x1 + 55, y1 - 10), cv2.FONT_ITALIC, 0.7, (0, 0, 0), 1)
        cv2.putText(image, f'{round(confidence, 2)}', (x1 + 150, y1 - 10), cv2.FONT_ITALIC, 0.7, (0, 0, 0), 1)
        cv2.putText(image, f'Wake-Up Drowsy!', (w//2-140, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


    if cls == 'drowsy':
        cv2.rectangle(image, (w // 2 - 150, 0), (w // 2 + 150, 50), (0, 0, 255), -1)
        cv2.putText(image, f'Wake-Up Drowsy!', (w//2-140, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
        if not pygame.mixer.music.get_busy():
            pygame.mixer.music.play()
    else:
        if pygame.mixer.music.get_busy():
            pygame.mixer.music.stop()

    cv2.imshow('image', image)
    if cv2.waitKey(1) == ord('o'):
        break

cv2.destroyAllWindows()
video.release()